<a href="https://colab.research.google.com/github/gafur55/Senior-Design/blob/main/senior_prj_data_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:

# Unzip the uploaded file
with zipfile.ZipFile('Strawberry-chobani.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/strawberry-chobani')

# Check if the images are extracted
os.listdir('/content/strawberry-chobani/Strawberry-chobani')


In [ ]:
!pip install pillow-heif
from pillow_heif import register_heif_opener

# Path to the folder containing the images
image_folder = '/content/strawberry-chobani/Strawberry-chobani'

# Loop through all images in the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".HEIC"):
        img_path = os.path.join(image_folder, filename)
        # Load the image using Pillow if it's a HEIC file
        if filename.endswith(".HEIC"):
            register_heif_opener() # this line will register support for HEIC
            img = Image.open(img_path)
            # Convert to RGB and then to a NumPy array that OpenCV can use
            img = img.convert('RGB')
            img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        else:
            # Load the image using OpenCV if it's a JPG file
            img = cv2.imread(img_path)

        # Check if the image was loaded successfully
        if img is not None:
          img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB for display

          # Display the image
          plt.imshow(img_rgb)
          plt.axis('off')
          plt.show()
        else:
          print(f"Failed to load image: {filename}")

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# Create ImageDataGenerator for augmentation
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment images from the folder
for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".HEIC"):
        img_path = os.path.join(image_folder, filename)
        # Load the image using Pillow if it's a HEIC file
        if filename.endswith(".HEIC"):
            register_heif_opener() # this line will register support for HEIC
            img = Image.open(img_path)
        img_array = tf.keras.preprocessing.image.img_to_array(img)  # Convert image to array
        img_array = img_array.reshape((1,) + img_array.shape)  # Reshape to add batch dimension

        # Generate and save augmented images
        i = 0
        for batch in datagen.flow(img_array, batch_size=1, save_to_dir='/content/strawberry-chobani/Strawberry-chobani', save_prefix='aug', save_format='HEIC'):
            i += 1
            if i > 5:  # Generate 5 augmented images per original image
                break
